In [1]:
import openai
import os
import pandas as pd
import time

openai.api_key = ""

# 0. 데이터 불러오기

In [2]:
data = pd.read_csv('sim_del_naver_news_data_v3.csv')
data

,UTC-date,UTC-time,title,content,press,link
0,2016-12-31,2016-12-31 22:00:00,한국 금융산업 위기… ‘우버 모멘트’ 를 넘어라,© News1저금리·저성장 전통 금융산업 위기4차 산업혁명...세계는 핀테크 혁명 ...,뉴스1,https://n.news.naver.com/mnews/article/421/000...
1,2017-01-01,2017-01-01 06:15:04,"英 파운드, 2016년 최악의 통화 ···'디지털통화' 비트코인 최고의 수익자산...",【뉴욕=정지원 특파원】 영국의 파운드가 2106년 최악의 통화로 기록됐다. 파운드는...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...
2,2017-01-01,2017-01-01 07:03:04,"[신년기획]블록체인, 금융권 넘어 IT기업도 관심",“블록체인은 인터넷 이후 나온 가장 혁신적이며 파괴적인 기술이다.”블록체인을 바라보...,39면 1단,https://n.news.naver.com/mnews/article/030/000...
3,2017-01-02,2017-01-02 09:47:22,공공기관·기업 대상으로 랜섬웨어 다량 유포…관계자 주의 필요,[아시아경제 박소연 기자]이스트소프트는 2일 국내 주요 국가 기관과 기업을 대상으로...,아시아경제,https://n.news.naver.com/mnews/article/277/000...
4,2017-01-02,2017-01-02 08:04:01,블록체인 등 최신 핀테크 기술 도입… `금융 한류` 일으키자,올 세계 은행 80% '블록체인' 도입 전망금융회사 거래 비용 약 30% 절감 예측...,4면 TOP,https://n.news.naver.com/mnews/article/029/000...
...,...,...,...,...,...,...
22204,2023-08-30,2023-08-30 02:13:01,"웨이브릿지, 빈터와 전통·가상자산 혼합형 전략 지수 발표",사진=웨이브릿지퀀트 기반 핀테크 기업 웨이브릿지가 글로벌 지수 빈터(Vinter)와...,한국경제,https://n.news.naver.com/mnews/article/015/000...
22205,2023-08-30,2023-08-30 02:14:01,"CJ ONE, 디지털 멤버십 NFT 발행",구매고객에 매달 최대 1만 포인트CJ올리브네트웍스가 운영하는 라이프스타일 멤버십 서...,16면 1단,https://n.news.naver.com/mnews/article/016/000...
22206,2023-08-30,2023-08-30 02:31:01,"‘신뢰’ 등에 업은 페이팔의 스테이블 코인, 시장의 메기가 될까[비트코인 A...",미국 캘리포니아주에 위치한 페이팔 본사./연합뉴스결제 공룡 페이팔이 스테이블 코인 ...,한경비즈니스,https://n.news.naver.com/mnews/article/050/000...
22207,2023-08-30,2023-08-30 01:50:07,"웨이브릿지, HAAM 전략 지수 개발…'코인 투자 손실률 최소화'",글로벌 지수 사업자 '빈터'와 맞손[ⓒ웨이브릿지][디지털데일리 박세아 기자] 퀀트 ...,디지털데일리,https://n.news.naver.com/mnews/article/138/000...


### model

In [3]:
# def get_completion(prompt, model="gpt-3.5-turbo"):
#     messages = [{"role": "user", "content": prompt}]
#     response = openai.ChatCompletion.create(
#         model=model,
#         messages=messages,
#         temperature=0,
#     )
#     return response.choices[0].message["content"]

In [3]:
def get_completion_16k(prompt, model="gpt-3.5-turbo-16k"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

def get_completion_with_delay(x):
    result = get_completion_16k(x)
    time.sleep(2)
    return result

In [5]:
# def get_completion_4(prompt, model="gpt-4"):
#     messages = [{"role": "user", "content": prompt}]
#     response = openai.ChatCompletion.create(
#         model=model,
#         messages=messages,
#         temperature=0,
#     )
#     return response.choices[0].message["content"]

# 1. 제목 + 본문 sentiment

In [43]:
prompt1 = "이전 지시사항은 모두 무시하십시오. 당신은 금융 전문가로 가정하며 금융자산 추천 경험이 있습니다. 만약 기사의 제목과 내용이 다음날 한국에서 거래되는 암호화폐의 가격에 좋은 소식이 확실하다면 1에 가까운 수를, 나쁜 소식이 확실하다면 –1에 가까운 수, 불확실하거나 암호화폐 가격에 대한 정보가 부족한 경우 0에 가까운 수로 답변해주세요. 다시 말해, 뉴스 기사 제목과 내용을 읽고 다음날 암호화폐 가격에 대해 좋은 소식인지, 아니면 나쁜 소식인지, 혹은 불확실한지를 –1과 1 사이의 실수로만 답하세요. 기사 내에 여러 개의 기사나 내용이 있다면 종합적으로 고려하여 여러개의 숫자가 아닌 하나의 숫자로만 답변하세요. 설명이나 추가적인 텍스트 없이 숫자만 답변하세요."

In [ ]:
# 결과를 저장할 빈 데이터프레임을 초기화합니다.
result1 = pd.DataFrame()

In [ ]:
# data의 모든 행을 100행씩 처리합니다.
for i in range(0, len(data), 100):
    subset = data.iloc[i:i+100]
    subset['score'] = (prompt1 + '제목:' + subset['title'] + '내용:' + subset['content']).apply(get_completion_with_delay)
    result1 = result1.append(subset)

result1.reset_index(drop=True, inplace=True)
result1

In [ ]:
result1.to_csv('result1.csv', encoding='utf-8-sig')

# 2. 제목 sentiment

In [46]:
prompt2 = "이전 지시사항은 모두 무시하십시오. 당신은 금융 전문가로 가정하며 금융자산 추천 경험이 있습니다. 만약 기사의 제목이 다음날 한국에서 거래되는 암호화폐의 가격에 좋은 소식이 확실하다면 1에 가까운 수를, 나쁜 소식이 확실하다면 –1에 가까운 수, 불확실하거나 암호화폐 가격에 대한 정보가 부족한 경우 0에 가까운 수로 답변해주세요. 다시 말해, 뉴스 기사 제목을 읽고 다음날 암호화폐 가격에 대해 좋은 소식인지, 아니면 나쁜 소식인지, 혹은 불확실한지를 –1과 1 사이의 실수로만 답하세요. 기사 내에 여러 개의 기사나 내용이 있다면 종합적으로 고려하여 여러개의 숫자가 아닌 하나의 숫자로만 답변하세요. 설명이나 추가적인 텍스트 없이 숫자만 답변하세요."

In [ ]:
# 결과를 저장할 빈 데이터프레임을 초기화합니다.
result2 = pd.DataFrame()

In [ ]:
# data의 모든 행을 100행씩 처리합니다.
for i in range(0, len(data), 100):
    subset = data.iloc[i:i+100]
    subset['score_title'] = (prompt2 + '제목:' + subset['title']).apply(get_completion_with_delay)
    result2 = result2.append(subset)

result2.reset_index(drop=True, inplace=True)
result2

In [ ]:
result2.to_csv('result2.csv', encoding='utf-8-sig')

# 3. 본문 sentiment

In [7]:
prompt3 = "이전 지시사항은 모두 무시하십시오. 당신은 금융 전문가로 가정하며 금융자산 추천 경험이 있습니다. 만약 기사의 내용이 다음날 한국에서 거래되는 암호화폐의 가격에 좋은 소식이 확실하다면 1에 가까운 수를, 나쁜 소식이 확실하다면 –1에 가까운 수, 불확실하거나 암호화폐 가격에 대한 정보가 부족한 경우 0에 가까운 수로 답변해주세요. 다시 말해, 뉴스 기사 내용을 읽고 다음날 암호화폐 가격에 대해 좋은 소식인지, 아니면 나쁜 소식인지, 혹은 불확실한지를 –1과 1 사이의 실수로만 답하세요. 기사 내에 여러 개의 기사나 내용이 있다면 종합적으로 고려하여 여러개의 숫자가 아닌 하나의 숫자로만 답변하세요. 설명이나 추가적인 텍스트 없이 숫자만 답변하세요."

In [ ]:
# 결과를 저장할 빈 데이터프레임을 초기화합니다.
result3 = pd.DataFrame()

In [ ]:
# data의 모든 행을 100행씩 처리합니다.
for i in range(0, len(data), 100):
    subset = data.iloc[i:i+100]
    subset['score_content'] = (prompt3 +'내용:' + subset['content']).apply(get_completion_with_delay)
    result3 = result3.append(subset)

result3.reset_index(drop=True, inplace=True)
result3

In [ ]:
result3.to_csv('result3.csv', encoding='utf-8-sig')